# RoboTutor Data Analysis

## Importing libraries

In [1]:
import pandas as pd
import sklearn
import numpy as np

In [2]:
from datetime import datetime
import dateutil

In [ ]:
import 

## Defining data folders

In [3]:
folder_data = '/home/amogh/Desktop/CMU/LearnLab_Summer_School/Data/'

In [4]:
folder_beta_sites_data = '/home/amogh/Desktop/CMU/LearnLab_Summer_School/Data/beta_sites_upto_27_july/'

In [5]:
path_data_beta_sites = folder_beta_sites_data + 'ds2577_tx_All_Data_4463_2018_0728_005503.txt'

## Loading raw data

In [6]:
!ls $folder_beta_sites_data

ds2577_tx_All_Data_4463_2018_0728_005503.txt
README.txt
RobotutorDatashopUploadDocumentation_20180727.xlsx
TERMS.html


In [7]:
df_data_beta_sites = pd.read_csv(path_data_beta_sites, sep = '\t')

/home/amogh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,75,76,79,82,83,84,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Visualising data**

In [8]:
df_data_beta_sites.head()

,Row,Sample Name,Transaction Id,Anon Student Id,Session Id,Time,Time Zone,Duration (sec),Student Response Type,Student Response Subtype,...,CF (Student School),CF (Student Used Scaffold),CF (Tablet Name),CF (Tablet Number),CF (Total Activity Problems),CF (Tutor Sequence Session),CF (Tutor Sequence User),CF (Unix Epoch),CF (User Id Order),CF (Version)
0,1,All Data,556a357f1676e9f396b472affacf1052,20180421_081922-0-52000266fecfa4e5,52000266fecfa4e5_20180503_095852,2018-05-03 12:59:15,NaN,0,NaN,NaN,...,NaN,NaN,3,52000266fecfa4e5,5.0,1,1,1.525356e+12,2,sw.1.8.9.1
1,2,All Data,19b3b667ff8f23ef0eff507504b00b7e,20180421_081922-0-52000266fecfa4e5,52000266fecfa4e5_20180503_095852,2018-05-03 12:59:20,NaN,5,NaN,NaN,...,NaN,NaN,3,52000266fecfa4e5,5.0,1,1,1.525356e+12,3,sw.1.8.9.1
2,3,All Data,8e06a2e8593ad9e657942d8bed372ee7,20180421_081922-0-52000266fecfa4e5,52000266fecfa4e5_20180503_095852,2018-05-03 12:59:24,NaN,4,NaN,NaN,...,NaN,NaN,3,52000266fecfa4e5,5.0,1,1,1.525356e+12,4,sw.1.8.9.1
3,4,All Data,84127ffce7384bbbb55d6075633012ac,20180421_081922-0-52000266fecfa4e5,52000266fecfa4e5_20180503_095852,2018-05-03 12:59:28,NaN,4,NaN,NaN,...,NaN,NaN,3,52000266fecfa4e5,5.0,1,1,1.525356e+12,5,sw.1.8.9.1
4,5,All Data,a9afc7bf3339a253b07e1ae593a9663b,20180421_081922-0-52000266fecfa4e5,52000266fecfa4e5_20180503_095852,2018-05-03 12:59:55,NaN,27,NaN,NaN,...,NaN,NaN,3,52000266fecfa4e5,9.0,2,2,1.525356e+12,6,sw.1.8.9.1


In [9]:
list_columns_df_beta_sites = df_data_beta_sites.columns.values
list_columns_df_beta_sites

array(['Row', 'Sample Name', 'Transaction Id', 'Anon Student Id',
       'Session Id', 'Time', 'Time Zone', 'Duration (sec)',
       'Student Response Type', 'Student Response Subtype',
       'Tutor Response Type', 'Tutor Response Subtype',
       'Level (Tutor Name)', 'Level (Tutor)', 'Problem Name',
       'Problem View', 'Problem Start Time', 'Step Name',
       'Attempt At Step', 'Is Last Attempt', 'Outcome', 'Selection',
       'Action', 'Input', 'Feedback Text', 'Feedback Classification',
       'Help Level', 'Total Num Hints', 'KC (Tutor)',
       'KC Category (Tutor)', 'KC (Tutor Problem Name)',
       'KC Category (Tutor Problem Name)', 'KC (Village)',
       'KC Category (Village)', 'KC (Matrix)', 'KC Category (Matrix)',
       'KC (Order)', 'KC Category (Order)', 'KC (Channel)',
       'KC Category (Channel)', 'KC (Option Behavior)',
       'KC Category (Option Behavior)', 'KC (Content)',
       'KC Category (Content)', 'KC (Content By Tutor)',
       'KC Category (Content 

Target - To get the session frequency and the session duration for all the villages.

In [10]:
list_villages = list(set(df_data_beta_sites['KC (Village)']))
list_villages

['Mugeta', 'Bagamoyo']

## Helper functions (getDateAndTime, getTotalDuration, getTotalUseDays, getTotalUsableDays)

In [11]:
# to get data and time in a tuple from a common string(found in the Time column of the dataframe)
def getDateAndTime(datetime):
    
    """
    Splits a string with date and time and returns them as a tuple
    """
    
    split_list = datetime.split(' ')
    time = split_list[1]
    date = split_list[0]
    return date, time

In [21]:
def getTotalDuration(df_data, village, student_id):
    
    """
    Input: data dataframe, village name, student_id
    Returns the total amount of usage in seconds for the tutor across all sessions in the data for a particular student in a village.
    """
    
    df_student = df_data[(df_data['KC (Village)'] == village) & (df_data['Anon Student Id'] == student_id)]
    list_sessions = list(set(df_student['Session Id']))
    sess_dur_sec = 0
    for sess in list_sessions:
        df_sess = df_student [df_student['Session Id'] == sess]
        DT = '%H:%M:%S'
        list_times = [dt[1] for dt in list(df_sess['Time'].apply(getDateAndTime))]
        list_times = sorted(list_times, key=lambda x: datetime.strptime(x, DT))
        diff_time = datetime.strptime(list_times[-1],DT) - datetime.strptime(list_times[0],DT)
        sess_dur_sec += diff_time.seconds
#     print("the tutor was used for {} seconds which is {} minutes".format(sess_dur_sec,sess_dur_sec/60))        
    return sess_dur_sec


In [13]:
def getTotalUseDays(df_data, village, student_id):
    
    """
    Input: data dataframe, village name, student_id, function to separate date and time
    Returns the total number of days tutor was used for a particular student in a village.
    """
    
    df_student = df_data[(df_data['KC (Village)'] == village) & (df_data['Anon Student Id'] == student_id)]
    list_dates = list(set(df_student['CF (Date)']))
    no_use_days = len(list_dates)
#     print("the tutor was used for {} days".format(no_use_days))    
    return no_use_days

In [23]:
def getTotalUsableDays(df_data, village, student_id):
    
    """
    Input: data dataframe, village name, student_id
    Returns the total number of days tutor could've been used for a particular student in a particular village.
    """
    
    df_student = df_data[(df_data['KC (Village)'] == village) & (df_data['Anon Student Id'] == student_id)]
    list_dates = list(set(df_student['CF (Date)']))
    list_dates = sorted(list_dates, key=lambda x:datetime.strptime(x, "%Y-%m-%d"))
    first_use_date = list_dates[0]
    last_use_date = list_dates[-1]
    no_usable_days = 1 + (datetime.strptime(last_use_date,"%Y-%m-%d") - datetime.strptime(first_use_date,"%Y-%m-%d")).days
#     print("the tutor was usable over {} days".format(no_usable_days))    
    return no_usable_days

## Functions to get avg session duration and average session frequency

In [24]:
def getAvgSessionDuration(df_data, village, student_id):
    
    """
    Input: data, village name, student id
    Returns the avg session duration obtained by dividing total use time by no of days that the tutor was used by a particular student in a particular village
    """
    
    total_duration = getTotalDuration(df_data, village, student_id)
    total_use_days = getTotalUseDays(df_data, village, student_id)
    avg_session_duration = total_duration/total_use_days
    return avg_session_duration

In [25]:
def getAvgSessionFrequency(df_data, village, student_id):

    """
    Input: data, village name, student id
    Returns the avg session frequency obtained by dividing total number of days used divided by number of days it could have been used by a particular student in a particular village
    """
    
    total_days_used = getTotalUseDays(df_data, village, student_id)
    total_days_usable = getTotalUsableDays(df_data, village, student_id)
    if(total_days_usable) == 0:
        print(village, student_id)
    avg_session_frequency = total_days_used / total_days_usable
    return avg_session_frequency

In [26]:
def getDurationFrequencyInVillage(df_data, village):
    list_children = list(set(df_data[df_data['KC (Village)'] == village] ['Anon Student Id']))
    print("Village {} has {} student ids".format(village, len(list_children)))
    list_avg_session_duration = [getAvgSessionDuration(df_data, village, student_id) for student_id in list_children]
    list_avg_session_frequency = [getAvgSessionFrequency(df_data, village, student_id) for student_id in list_children]
    return list_avg_session_duration, list_avg_session_frequency

## Calculating Metrics (Mean, variance)

In [35]:
def getUsageMetrics(df_data, village):
    
    """
    Takes in the dataframe for the data and village name, calculates metrics (mean, std) for the village
    """
    
    list_avg_session_duration, list_avg_session_frequency = getDurationFrequencyInVillage(df_data, village)
    array_avg_session_duration_minutes = np.array(list_avg_session_duration)/60
    array_avg_session_frequency = np.array(list_avg_session_frequency)
    mean_session_duration = np.mean(array_avg_session_duration_minutes)
    mean_session_frequency = np.mean(array_avg_session_frequency)
    std_session_duration = np.std(array_avg_session_duration_minutes)
    std_session_frequency = np.std(array_avg_session_frequency)
    print(mean_session_duration, mean_session_frequency, std_session_duration, std_session_frequency)

In [36]:
getTotalUsableDays(df_data_beta_sites, "Bagamoyo", "20180428_102608-3-52009f57fe7e943d")

2

In [37]:
getUsageMetrics(df_data_beta_sites, 'Bagamoyo')

Village Bagamoyo has 109 student ids
32.43659531090724 0.843391774355077 14.99810436352205 0.30034522444222894


### Questions to ask

1. CF (duration) is in seconds, does it make sense?